In [ ]:
#Refrence https://www.analyticsvidhya.com/blog/2021/06/language-translation-with-transformer-in-python/

!pip install sacrebleu
!pip install rouge

In [ ]:
import math
import torchtext
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from collections import Counter
# from torchtext.vocab import Vocab

from torchtext.vocab import vocab as voc
from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn import (TransformerEncoder, TransformerDecoder,TransformerEncoderLayer, TransformerDecoderLayer)
import io
import time
import pandas as pd
TRAIN_LIMIT = 1000
from sacrebleu.metrics import BLEU

In [ ]:
AVG = 219
def get_dataset():
    fil = open('./processed_task_2_event_summarization_train.tsv')
    lines = fil.readlines()
    data = []
    for line in lines:
      [left,right] = line.split('\t')
      
      if len(right)<AVG:
        data.append(line.split('\t')) 
    train_df = pd.DataFrame(data, columns=[ "input_text", "target_text"])
    fil = open('./processed_task_2_event_summarization_valid.tsv')
    lines = fil.readlines()
    data = []
    lens = 0
    for line in lines:
      [left,right] = line.split('\t')
      
      # if len(right)<AVG:
      data.append(line.split('\t')) 
    
    valid_df = pd.DataFrame(data, columns=[ "input_text", "target_text"])
    return train_df,valid_df

In [ ]:

train,valid = get_dataset()
train = train[:TRAIN_LIMIT]
len(train)

# train = train[:TRAIN_LIMIT]
# valid = valid
def get_training_corpus():
  for i in range(len(train)):
    line = train[i]
    print(line['target_text'])
    yield line['target_text'].split()
  for i in range(len(valid)):
    line = valid[i]
    yield line['target_text'].split()

tokenizer = get_tokenizer("basic_english")

def build_vocab(filepath, tokenizer):
  counter = Counter()
  for file in filepath:
    with io.open(file, encoding="utf8") as f:
      for string_ in f:
        counter.update(tokenizer(string_))
  return voc(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

In [ ]:
vocab = build_vocab(['processed_task_2_event_summarization_train.tsv','processed_task_2_event_summarization_valid.tsv'], tokenizer)

vocab.get_stoi()['test']


def data_process(train):
  data=[]
  for i in range(len(train)):      
      input_text_tensor_ = torch.tensor([vocab[token] for token in tokenizer(train['input_text'][i])],dtype=torch.long)
      target_text_tensor_ = torch.tensor([vocab[token] for token in tokenizer(train['target_text'][i])],dtype=torch.long)
      data.append((input_text_tensor_, target_text_tensor_))
  return data

train_data = data_process(train)
valid_data = data_process(valid)

len(train_data)

PAD_IDX = vocab['<pad>']
BOS_IDX = vocab['<bos>']
EOS_IDX = vocab['<eos>']


In [ ]:
def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch
BATCH_SIZE = 2
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(valid_data, batch_size=BATCH_SIZE,shuffle=True, collate_fn=generate_batch)


In [ ]:

class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory,tgt_mask)

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask



In [ ]:

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
SRC_VOCAB_SIZE = len(vocab)
TGT_VOCAB_SIZE = len(vocab)
EMB_SIZE = 512
# NHEAD = 8
# FFN_HID_DIM = 512
# BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 5
NUM_DECODER_LAYERS = 5
# NUM_EPOCHS = 50
NUM_EPOCHS = 50
NHEAD = 8
# NUM_ENCODER_LAYERS = 1
# NUM_DECODER_LAYERS = 1
BATCH_SIZE = 2
# EMB_SIZE = 256
FFN_HID_DIM = 256
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

def train_epoch(model, train_iter, optimizer):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_iter):
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                       src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    torch.save(model, './test')

    return losses / len(train_iter)


def evaluate(model, val_iter):
    model.eval()
    losses = 0
    for idx, (src, tgt) in (enumerate(valid_iter)):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                       src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(val_iter)


def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys



In [ ]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    train_loss = train_epoch(transformer, train_iter, optimizer)
    end_time = time.time()
    val_loss = evaluate(transformer, valid_iter)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
           f"Epoch time = {(end_time - start_time):.3f}s"))


Epoch: 1, Train loss: 6.862, Val loss: 6.213, Epoch time = 74.052s
Epoch: 2, Train loss: 5.385, Val loss: 5.938, Epoch time = 72.659s
Epoch: 3, Train loss: 4.969, Val loss: 5.814, Epoch time = 72.995s
Epoch: 4, Train loss: 4.685, Val loss: 5.647, Epoch time = 72.872s
Epoch: 5, Train loss: 4.465, Val loss: 5.491, Epoch time = 72.742s
Epoch: 6, Train loss: 4.274, Val loss: 5.467, Epoch time = 72.568s
Epoch: 7, Train loss: 4.108, Val loss: 5.361, Epoch time = 72.611s
Epoch: 8, Train loss: 3.964, Val loss: 5.366, Epoch time = 72.655s


In [ ]:
def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
    model.eval()
    tokens = [BOS_IDX] + [src_vocab.get_stoi()[tok] for tok in src_tokenizer(src)] + [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1))
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    
    return " ".join([tgt_vocab.get_itos()[int(tok)] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")


# for i in range(10):
#   output = translate(transformer, valid['input_text'][i], vocab, vocab, tokenizer)
#   print('---------BATCH',i+1,'----------')
#   print(valid['input_text'][i])
#   print('<start> ',output,' <end>')
#   print(valid['target_text'][i])
#   print('\n')

for i in range(10):
  output = translate(transformer, valid['input_text'][i], vocab, vocab, tokenizer)
  print('---------BATCH',i+1,'----------')
  print(valid['input_text'][i])
  print('<start> ',output,' <end>')
  print(valid['target_text'][i])
  print('\n')



In [ ]:
%pip install rouge

In [ ]:
from torchtext.data.metrics import bleu_score
from rouge import Rouge
rouge = Rouge()
candidate_corpus = []
references_corpus = []
for i in range(10):
  input,target = valid['input_text'][i],valid['target_text'][i]
  output = translate(transformer, valid['input_text'][i], vocab, vocab, tokenizer)
  input_words = input.split()
  target_words = target.split()
  output_words = output.split()
  candidate_corpus.append(target_words)
  references_corpus.append([output_words])
print('bleu_score',bleu_score(candidate_corpus, references_corpus))
cp = []
rp = []
for c,r in zip(candidate_corpus, references_corpus):
  cp.append(" ".join(c))
  rp.append(" ".join(r[0]))
scores = rouge.get_scores(cp, rp, avg=True)
print('rouge',scores)